# Deriving the determining equations of the Lotka-Volterra model
*Date*: 2022-04-13,<br>
*Written by:* Johannes Borgqvist.<br>
Ok, so consider the dimensionless *Lotka-Volterra* model:
\begin{align}
    \dfrac{\mathrm{d}u}{\mathrm{d}t}&=u(1-v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}t}&=\alpha v(u-1).\\
\end{align}
Now, we would like to see if we can derive the determining equations from this using sympy. <br>

In [1]:
# Import sympy
from sympy import *
# Translate a string to symbolic expression
from sympy.parsing.sympy_parser import parse_expr
# Finding monomials
from sympy.polys.monomials import itermonomials, monomial_count
# Ordering monomials 
from sympy.polys.orderings import monomial_key

# Setting up the reaction terms and so on

In [2]:
# Independent variable
t = symbols('t')
# Dependent variables
u,v = symbols('u v',cls=Function)
# Define our parameter a
a = symbols('a')
# Define the states
states = [u(t), v(t)]
# Define our reaction terms
omega_list = [u(t)*(1-v(t)), a*v(t)*(u(t)-1)]

# Function for setting up the linearised symmetry condition
So now we will try to set up a general function for deriving the linearised symmetry conditions!

In [3]:
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 1: "lin_sym_conds_fibre_preserving"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This functions sets up the linearised symmetry conditions
# given an independent variable t, the states and the reaction terms
def lin_sym_conds_fibre_preserving(t,states,reaction_terms):
    # Temporary variables for the dependent variables so that we can 
    # differentiate the reaction terms with respect to each state
    x = symbols('x')    
    # Also, we define a kind of dummy variable
    _xi_1 = symbols('_xi_1')
    # Add a temporary function which we differentiate all the time
    f = symbols('f',cls=Function)
    # Define a list of tangents
    tangent_list = []
    # Define the time tangent
    xi = symbols('xi', cls=Function)
    # Add it to the tangent list
    tangent_list.append(xi(t))
    # Loop over the states and add a tangent for each state, hey?
    for state_index in range(1,len(states)+1):
        # Here, we essentially define a string which we then convert
        # to a symbolic expression using "parse_expr"
        temp_str = "eta_" + str(state_index) + "(" + str(t) + "," 
        for state in states:
            temp_str += str(state) + ","
        temp_str += ")"
        temp_str = temp_str.replace(",)",")")
        # Add our new tangent
        tangent_list.append(parse_expr(temp_str))
    # Ok, so at this point we have our tangent lists, now we can start assembling our linearised symmetry conditions
    # Allocate memory for our linearised symmetry conditions
    lin_sym_original = []
    # Loop over the number of states and define the linearised symmetry condition
    for state_index in range(1,len(states)+1):
        # Define the LHS
        LHS = Derivative(tangent_list[state_index],t).doit() - omega_list[state_index-1]*Derivative(tangent_list[0],t).doit()
        # Allocate memory for the RHS
        RHS = 0
        # Add all partial derivatives of the current reaction term by looping over all states and times
        for RHS_index in range(1,len(states)+1):
            # Extract temporary tangent
            tangent_temp = tangent_list[RHS_index]
            # Partial derivative wrt to the time
            RHS += (Derivative(omega_list[state_index-1].subs(states[RHS_index-1],x),x).subs(x,states[RHS_index-1]).doit()*tangent_temp)
        # After this is done we loop over the states one more time and substitute all time derivatives of the
        # states with the reaction terms
        eq_temp = LHS - RHS
        for state_index in range(len(omega_list)):
            # Also, we substitute the reaction terms in the LHS
            eq_temp = eq_temp.subs(Derivative(states[state_index],t),omega_list[state_index])
        # Append the equation defined as LHS-RHS
        lin_sym_original.append((eq_temp,0))
    # Return the list with all the tangents
    return tangent_list, lin_sym_original
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 2: "latex_printing_of_lin_sym"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function takes in the time t, the states, tangent_list and the linearised symmetry condition
# and prints them in a nice fashion.
def latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original):
    # We essentially return a giant string which can be printed, so let's start constructing it
    tangent_str = "\n\nThe tangents are:\n$$" + latex(tangent_list) + "$$\n"
    output_str = "\n\\begin{align*}\n"
    # Get the arguments as well
    argument_str = latex(tangent_list[1]).replace("\\eta_{1}","")
    # Loop through the linearised symmetry condition and add the equations
    for lin_sym in lin_sym_original:
        output_str += latex(lin_sym[0]) + "&=" + latex(lin_sym[1]) + ",\\\\\n"
    output_str += "\\end{align*}\n\n"
    # Now we replace the arguments as well to see if this helps
    output_str = output_str.replace(argument_str,"")  
    # Remove time dependence already
    output_str = output_str.replace("{\\left(t \\right)}","")    
    # Loop through the tangents and replace strings for partial derivatives
    for tangent_index in range(len(tangent_list)):
        if tangent_index == 0:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\xi{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{d\\xi}{dt}")
            output_str = output_str.replace("\\frac{d}{d t} \\xi","\\frac{d\\xi}{dt}")
        else:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\eta_{" + str(tangent_index) + "}{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{\\partial\\eta_"+ str(tangent_index)+"}{\\partial t}")            
            state_str = ""
            # Loop through the states and find the partial derivatives
            for state_index in range(len(states)):
                # Extract the string for the state at hand
                state_str = latex(states[state_index])
                state_str = state_str.replace("{\\left(t \\right)}","")
                # Replace the partial derivative at hand
                output_str = output_str.replace("\\frac{d}{d " + state_str +"} \\eta_{"+ str(tangent_index) +"}","\\frac{\\partial\\eta_{"+ str(tangent_index) +"}}{\\partial "+ state_str +"}")            
    # Return the output string
    return tangent_str,output_str
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 3: "create_tangent_ansatze"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# The function takes three inputs:
#1. The time variable t,
#2. The states in the list states, 
#2. The degree of the polynomial ("degree_polynomial").
# It returns two outputs:
# 1.The variables (independent and dependent) in the symmetry framework,
# 2. A list of all tangents ("eta_list").
# The function uses various functionalities in sympy and the polynomials are generated using the built in functions in the polys.monomial part of the sympy library. To automate the generation of the sympy functions and symbol, the "exec" command is used at multiple places in the function.
def create_tangent_ansatze(t,states,degree_polynomial):
    # Calculate the number of variables and number of states
    num_of_states = len(states)
    num_of_variables = 1
    # Allocate our to lists which will be the output:
    x = [] # The independent and dependent variables
    eta_list = [] # The function with the tangents        
    c = [] # The list of the coefficients
    # Allocate the polynomials which we will return as well
    polynomial_list = []
    #----------------------------------------------------------------------------------
    # STEP 1: A LIST OF ALL INDEPENDENT AND DEPENDENT VARIABLES
    #----------------------------------------------------------------------------------
    substitution_list = [t]
    for state in states:
        substitution_list.append(state)
    #----------------------------------------------------------------------------------
    # STEP 2: GENERATE TEMPORARY VARIABLES    
    #----------------------------------------------------------------------------------
    # Add variables to the list
    for index in range(num_of_states+num_of_variables):
        # Allocate variable
        exec("x_%d = Symbol(\'x_%d\') "%(index,index))  
        #parse_expr("x_" + str(index) + " = Symbol(\'x_" + str(index) + "\')")
        # Add variable to the list    
        #parse_expr("x.append(x_" + str(index) + ")")
        exec("x.append(x_%d)"%(index)) 
    #----------------------------------------------------------------------------------
    # STEP 3: GENERATE POLYNOMIALS IN THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Generate all monomials for our polynomials
    M = list(itermonomials(x, degree_polynomial))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', x))
    # Calculate the number of terms in each of the polynomials
    num_of_terms = monomial_count(num_of_states+num_of_variables,degree_polynomial)
    #----------------------------------------------------------------------------------
    # STEP 4: DEFINE THE TANGENTIAL ANSÄTZE IN TERMS OF THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Loop over the variables and create our tangent eta for each variable
    # and add it to the eta_list. Lastly, reset it to zero and start over 
    # again. 
    # Loop over our tangents
    for tangent_index in range(1,num_of_states+1):    
        # Allocate memory for our tangent
        exec("f_%d = symbols(\'f_%d\', cls=Function) "%(tangent_index,tangent_index)) 
        # Allocate memory for our function
        exec("f_%d = 0"%(tangent_index))
        # Allocate a temporary list for the two polynomials
        temp_list_polynomials = []
        # Loop over our two polynomials which define our ansatz
        for polynomial_index in range(1,3):
            # Define our polynomial at hand 
            exec("P_%d_%d = symbols(\'P_%d_%d\', cls=Function) "%(tangent_index,polynomial_index,tangent_index,polynomial_index)) 
            # Allocate memory for our newly created polynomial
            exec("P_%d_%d=0"%(tangent_index,polynomial_index))
            # Loop over the monomials and add them to the polynomial 
            for index in range(num_of_terms):   
                # Define a temporary index with the correct ordering
                temp_index = ((polynomial_index-1)*num_of_terms+index) + 2*((tangent_index-1)*num_of_terms)
                # Allocate a coefficient
                exec("c_%d = symbols(\'c_%d\') "%(temp_index,temp_index)) 
                # Add this coefficient to the set of unknowns
                exec("c.append(c_%d)"%(temp_index))
                # Add this coefficent to our polynomial at hand
                exec("P_%d_%d+=c_%d*M[%d]" % (tangent_index,polynomial_index,temp_index,index))
            # Add our polynomial to our temporary list of polynomials
            exec("temp_list_polynomials.append(P_%d_%d)"%(tangent_index,polynomial_index))
            # Add the polynomial to our function
            if polynomial_index == 1:
                exec("f_%d += P_%d_%d"%(tangent_index,tangent_index,polynomial_index))
            elif polynomial_index == 2:
                exec("f_%d = f_%d*exp(P_%d_%d)"%(tangent_index,tangent_index,tangent_index,polynomial_index))
        # Save our polynomials now
        polynomial_list.append(temp_list_polynomials)    
        # Lastly, save our function
        exec("eta_list.append(f_%d)"% (tangent_index))
    #----------------------------------------------------------------------------------
    # STEP 5: TRANSLATE BACK TO THE ORIGINAL VARIABLES
    #----------------------------------------------------------------------------------        
    # Loop over all independent and dependent variables
    for variable_index in range(len(x)):
        # Loop over the monomials and conduct the substitutions
        for index in range(num_of_terms):
            # Substitute variable    
            M[index] = M[index].subs(x[variable_index],substitution_list[variable_index])
        # Loop over all tangents and do the substitutions
        for tangent_index in range(len(eta_list)):
            # Substitute variable
            eta_list[tangent_index] = eta_list[tangent_index].subs(x[variable_index],substitution_list[variable_index])
            # Same goes for polynomials
            polynomial_list[tangent_index][0] = polynomial_list[tangent_index][0].subs(x[variable_index],substitution_list[variable_index])
            polynomial_list[tangent_index][1] = polynomial_list[tangent_index][1].subs(x[variable_index],substitution_list[variable_index])
    # Return the output    
    return c, eta_list, M, polynomial_list

# Test output from automated function

In [4]:
# Formulate the original linearised symmetry conditions and the list of tangents
tangent_list,lin_sym_original = lin_sym_conds_fibre_preserving(t,states,omega_list)    

In [5]:
# Create a nice output string for the original linearised symmetry conditions
tangent_str,lin_sym_original_str = latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original)
# Prompt to the user
print("The original linearised symmetry conditions are:")
print(lin_sym_original_str)

The original linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0,\\
\end{align*}




The original linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0,\\
\end{align*}

# Create tangential ansatze
Next, we want to test some polynomial ansatze to see if we can possibly simplify the linearised symmetry conditions. Since we already have tried the polynomials ansatze and these failed, we must modify these ansatze 

In [7]:
# Define the degree of the polynomial
degree_polynomial = 2
# Calculate the ansatze
c, eta_list, M, polynomial_list = create_tangent_ansatze(t,states,degree_polynomial)
# Print the degree of the polynomials and the unknown coefficients
print("We use ansatze of the type $$\\eta_i=P_{i,1}*\\exp\\left(P_{i,2}\\right)$$ for some index $i\\in\\left\\{1,\\ldots,n\\right\\}$ where $n$ is the number of states.")
print("Here, our two polynomials $P_{i,1}$ and $P_{i,2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $%d$."%(degree_polynomial))
print("Moreover, we have the following states or dependent variables:\n%s\nand one independent variable $t$."%(latex(Matrix(len(states),1,states),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\n\\end{equation}")))
print("Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:")
print(latex(Matrix(len(M),1,M),mode='equation').replace("\\begin{equation}","\\begin{equation}\nM=").replace("\\end{equation}",".\n\\end{equation}"))
print("Furthermore, we have the following unknown coefficients:")
print(latex(Matrix(len(c),1,c),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
print("Now, given all of these unknowns and coefficient, this is the ansatze we get:")
print("\\begin{align*}")
for eta_index in range(len(eta_list)):
    temp_str = "\\eta_{%d}&=" + latex(eta_list[eta_index]) 
    if eta_index == len(eta_list)-1:
        temp_str += ".\\\\"
    else:
        temp_str += ",\\\\"        
    print(temp_str % (eta_index+1))
print("\\end{align*}")

We use ansatze of the type $$\eta_i=P_{i,1}*\exp\left(P_{i,2}\right)$$ for some index $i\in\left\{1,\ldots,n\right\}$ where $n$ is the number of states.
Here, our two polynomials $P_{i,1}$ and $P_{i,2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $2$.
Moreover, we have the following states or dependent variables:
\begin{equation}
\left[\begin{matrix}u{\left(t \right)}\\v{\left(t \right)}\end{matrix}\right]
\end{equation}
and one independent variable $t$.
Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:
\begin{equation}
M=\left[\begin{matrix}1\\v{\left(t \right)}\\v^{2}{\left(t \right)}\\u{\left(t \right)}\\u{\left(t \right)} v{\left(t \right)}\\u^{2}{\left(t \right)}\\t\\t v{\left(t \right)}\\t u{\left(t \right)}\\t^{2}\end{matrix}\right].
\end{equation}
Furthermore, we have the following unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{mat

We use ansatze of the type $$\eta_i=P_{i,1}*\exp\left(P_{i,2}\right)$$ for some index $i\in\left\{1,\ldots,n\right\}$ where $n$ is the number of states.
Here, our two polynomials $P_{i,1}$ and $P_{i,2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $2$.
Moreover, we have the following states or dependent variables:
\begin{equation}
\left[\begin{matrix}u{\left(t \right)}\\v{\left(t \right)}\end{matrix}\right]
\end{equation}
and one independent variable $t$.
Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:
\begin{equation}
M=\left[\begin{matrix}1\\v{\left(t \right)}\\v^{2}{\left(t \right)}\\u{\left(t \right)}\\u{\left(t \right)} v{\left(t \right)}\\u^{2}{\left(t \right)}\\t\\t v{\left(t \right)}\\t u{\left(t \right)}\\t^{2}\end{matrix}\right].
\end{equation}
Furthermore, we have the following unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{28}\\c_{29}\\c_{30}\\c_{31}\\c_{32}\\c_{33}\\c_{34}\\c_{35}\\c_{36}\\c_{37}\\c_{38}\\c_{39}\end{matrix}\right].
\end{equation}
Now, given all of these unknowns and coefficient, this is the ansatze we get:
\begin{align*}
\eta_{1}&=\left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right) e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}},\\
\eta_{2}&=\left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right) e^{c_{30} + c_{31} v{\left(t \right)} + c_{32} v^{2}{\left(t \right)} + c_{33} u{\left(t \right)} + c_{34} u{\left(t \right)} v{\left(t \right)} + c_{35} u^{2}{\left(t \right)} + c_{36} t + c_{37} t v{\left(t \right)} + c_{38} t u{\left(t \right)} + c_{39} t^{2}}.\\
\end{align*}

# Substituting these ansatze into the linearised symmetry conditions
Let's try to substitute these ansatze into the linearised symmetry conditions.


In [20]:
# Define a temporary dummy variable
_xi_1 = symbols('_xi_1')
# Calculate the linearised symmetry condition
lin_sym_subs = []
for lin_sym in lin_sym_original:
    temp_eq = lin_sym[0]
    for index in range(len(eta_list)):
        temp_eq = temp_eq.subs(tangent_list[index+1],eta_list[index]).doit()
    lin_sym_subs.append(temp_eq)
# Substitue the time derivative as well
for lin_sym_index in range(len(lin_sym_subs)):
    lin_sym_subs[lin_sym_index] = simplify(lin_sym_subs[lin_sym_index].subs(Subs(Derivative(tangent_list[lin_sym_index+1].subs(t,_xi_1), _xi_1), _xi_1, t),Subs(Derivative(eta_list[lin_sym_index].subs(t,_xi_1), _xi_1), _xi_1, t)).doit()) 
# Now we can print these conditions to se what's up!
print("Linearised symmetry condition after substitution:")
print("\\begin{align*}")
for lin_sym_index in range(len(lin_sym_subs)):
    temp_str =  latex(lin_sym_subs[lin_sym_index]) + "&=0"
    if lin_sym_index == len(lin_sym_subs)-1:
        temp_str += ".\\\\"
    else:
        temp_str += ",\\\\"        
    print(temp_str)
print("\\end{align*}")


Linearised symmetry condition after substitution:
\begin{align*}
a \left(u{\left(t \right)} - 1\right) \left(c_{1} + 2 c_{2} v{\left(t \right)} + c_{4} u{\left(t \right)} + c_{7} t + \left(c_{11} + 2 c_{12} v{\left(t \right)} + c_{14} u{\left(t \right)} + c_{17} t\right) \left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right)\right) v{\left(t \right)} e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}} - \left(v{\left(t \right)} - 1\right) \left(c_{3} + c_{4} v{\left(t \right)} + 2 c_{5} u{\left(t \right)} + c_{8} t + \left(c_{13} + c_{14} v{\left(t \right)} + 2 c_{15} u{\le

Linearised symmetry condition after substitution:
\begin{align*}
a \left(u{\left(t \right)} - 1\right) \left(c_{1} + 2 c_{2} v{\left(t \right)} + c_{4} u{\left(t \right)} + c_{7} t + \left(c_{11} + 2 c_{12} v{\left(t \right)} + c_{14} u{\left(t \right)} + c_{17} t\right) \left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right)\right) v{\left(t \right)} e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}} - \left(v{\left(t \right)} - 1\right) \left(c_{3} + c_{4} v{\left(t \right)} + 2 c_{5} u{\left(t \right)} + c_{8} t + \left(c_{13} + c_{14} v{\left(t \right)} + 2 c_{15} u{\left(t \right)} + c_{18} t\right) \left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right)\right) u{\left(t \right)} e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}} + \left(v{\left(t \right)} - 1\right) \left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right) e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}} + \left(v{\left(t \right)} - 1\right) u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + \left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right) u{\left(t \right)} e^{c_{30} + c_{31} v{\left(t \right)} + c_{32} v^{2}{\left(t \right)} + c_{33} u{\left(t \right)} + c_{34} u{\left(t \right)} v{\left(t \right)} + c_{35} u^{2}{\left(t \right)} + c_{36} t + c_{37} t v{\left(t \right)} + c_{38} t u{\left(t \right)} + c_{39} t^{2}} + \left. \frac{\partial}{\partial \xi_{1}} \eta_{1}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}&=0,\\
a \left(u{\left(t \right)} - 1\right) \left(c_{21} + 2 c_{22} v{\left(t \right)} + c_{24} u{\left(t \right)} + c_{27} t + \left(c_{31} + 2 c_{32} v{\left(t \right)} + c_{34} u{\left(t \right)} + c_{37} t\right) \left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right)\right) v{\left(t \right)} e^{c_{30} + c_{31} v{\left(t \right)} + c_{32} v^{2}{\left(t \right)} + c_{33} u{\left(t \right)} + c_{34} u{\left(t \right)} v{\left(t \right)} + c_{35} u^{2}{\left(t \right)} + c_{36} t + c_{37} t v{\left(t \right)} + c_{38} t u{\left(t \right)} + c_{39} t^{2}} - a \left(u{\left(t \right)} - 1\right) \left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right) e^{c_{30} + c_{31} v{\left(t \right)} + c_{32} v^{2}{\left(t \right)} + c_{33} u{\left(t \right)} + c_{34} u{\left(t \right)} v{\left(t \right)} + c_{35} u^{2}{\left(t \right)} + c_{36} t + c_{37} t v{\left(t \right)} + c_{38} t u{\left(t \right)} + c_{39} t^{2}} - a \left(u{\left(t \right)} - 1\right) v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} - a \left(c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}\right) v{\left(t \right)} e^{c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}} - \left(v{\left(t \right)} - 1\right) \left(c_{23} + c_{24} v{\left(t \right)} + 2 c_{25} u{\left(t \right)} + c_{28} t + \left(c_{33} + c_{34} v{\left(t \right)} + 2 c_{35} u{\left(t \right)} + c_{38} t\right) \left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right)\right) u{\left(t \right)} e^{c_{30} + c_{31} v{\left(t \right)} + c_{32} v^{2}{\left(t \right)} + c_{33} u{\left(t \right)} + c_{34} u{\left(t \right)} v{\left(t \right)} + c_{35} u^{2}{\left(t \right)} + c_{36} t + c_{37} t v{\left(t \right)} + c_{38} t u{\left(t \right)} + c_{39} t^{2}} + \left. \frac{\partial}{\partial \xi_{1}} \eta_{2}{\left(\xi_{1},u{\left(t \right)},v{\left(t \right)} \right)} \right|_{\substack{ \xi_{1}=t }}&=0.\\
\end{align*}